In [ ]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle
from sklearn.utils import parallel_backend

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp
from bayes_opt import BayesianOptimization

import vectorizer
import vectorizer 
importlib.reload(vectorizer)
from vectorizer import  WeightedTfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import  h2o

In [84]:
from h2o.estimators import H2OXGBoostEstimator, H2ODeepLearningEstimator,H2ORandomForestEstimator,H2OSupportVectorMachineEstimator

In [ ]:

h2o.init(max_mem_size='14G')

In [71]:
from hyperopt import fmin, tpe, hp,Trials

def xgb_cv(space):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'max_depth': int(space['max_depth']), 'ntrees': int(space['ntrees']),'min_rows':int(space['min_rows']),'eta':float(space['eta']),
              'learn_rate':float(space['learn_rate']),'sample_rate':float(space['sample_rate']),
              'colsample_bytree':float(space['colsample_bytree']),'reg_lambda':float(space['reg_lambda']),'reg_alpha':float(space['reg_alpha'])}
      
    estimator = H2OXGBoostEstimator(**params,nfolds=10)
    estimator.train(x=vocab,y='target',training_frame=train)
    return estimator.logloss()

def optimize_xgb():
    """Apply Bayesian Optimization to XGB parameters."""
    
    trials = Trials()
    space={"max_depth": hp.uniform('max_depth',4, 40), "ntrees": hp.uniform('x_ntrees',50,400),'min_rows':hp.uniform('x_min_rows',2,50),'eta':hp.uniform('x_eta',0.01,1),
                 'learn_rate':hp.uniform('x_learn_rate',0.01,1),'sample_rate':hp.uniform('x_sample_rate',0.1,0.5),'colsample_bytree':hp.uniform('x_colsample_bytree',0.2,0.8),
                 'reg_lambda':hp.uniform('x_reg_lambda',0,1),'reg_alpha':hp.uniform('x_reg_alpha',0,1)}
    best = fmin(fn=xgb_cv,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

    print("Final result:", best)
    


In [72]:
from hyperopt import fmin, tpe, hp,Trials

def svm_cv(space):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'gamma': float(space['gamma'])}
      
    estimator = H2OSupportVectorMachineEstimator(**params,max_iterations=400)
    estimator.train(x=vocab,y='target',training_frame=train)
    return estimator.logloss()

def optimize_svm():
    """Apply Bayesian Optimization to XGB parameters."""
    
    trials = Trials()
    space={"gamma": hp.uniform('x_gamma',0,1),}
    best = fmin(fn=svm_cv,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

    print("Final result:", best)

In [73]:
list((12,12))

[12, 12]

In [74]:
from hyperopt import fmin, tpe, hp,Trials

def rf_cv(space):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'max_depth': int(space['max_depth']), 
              'ntrees': int(space['ntrees']),'min_rows':int(space['min_rows']),
              'sample_rate':float(space['sample_rate'])}
      
    estimator = H2ORandomForestEstimator(**params,nfolds=10)
    estimator.train(x=vocab,y='target',training_frame=train,validation_frame=test)
    return estimator.logloss()

def optimize_rf():
    """Apply Bayesian Optimization to XGB parameters."""
    
    trials = Trials()
    space={"max_depth": hp.uniform('max_depth',4, 40), 
           "ntrees": hp.uniform('x_ntrees',50,400),
           'min_rows':hp.uniform('x_min_rows',2,50),
           'sample_rate':hp.uniform('x_sample_rate',0.1,0.5)}
    best = fmin(fn=rf_cv,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

    print("Final result:", best)

In [92]:
import joblib
vocab=joblib.load('../objects/20newsgroup/vocab.dtm.final')
newdtm =joblib.load('../objects/20newsgroup/newdtm.final')
corpusnormalizado = joblib.load('../data/processed/20newsgroup/corpus_normalizado.final')
dtm = newdtm.todense()
dtm_scaled = scale(dtm)

In [93]:
dtm_scaled = pd.DataFrame(data=dtm_scaled, columns=vocab) ##revisar que está bien generado, si no ir al paso anterior

In [94]:
dtm_scaled['target']=corpusnormalizado.iloc[:,1]


In [95]:
dtm_scaled.head()

,read,24,service,numb,ask,dept,question,news,support,17,...,great,lot,talk,little,happen,claim,doesn,opinion,leave,target
0,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,-0.332306,-0.297651,-0.254025,-0.239241,...,-0.277919,5.889076,-0.249082,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,1.0
1,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,-0.332306,-0.297651,2.721457,-0.239241,...,-0.277919,-0.293101,-0.249082,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,1.0
2,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,1.791602,-0.332306,-0.297651,-0.254025,-0.239241,...,4.392435,-0.293101,3.137025,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,0.0
3,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,-0.332306,-0.297651,-0.254025,-0.239241,...,-0.277919,-0.293101,0.846980,-0.275675,-0.262523,-0.223794,0.719420,-0.27425,-0.260799,1.0
4,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,1.677027,-0.297651,-0.254025,-0.239241,...,-0.277919,-0.293101,-0.249082,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,1.0


In [96]:
corpusnormalizado.head()

,cleaned_text,new_target
0,organization post office carnegie mellon pittsburgh pa line 12 nnt...,1.0
1,summary seek recommendation vlb video card nntp post host midway.e...,1.0
2,line 95 nntp post host viktoria.dsv.su.se organization dept comput...,0.0
3,originator guyd pal500.austin.ibm.com organization ibm austin line...,1.0
4,organization sophomore mechanical engineer carnegie mellon pittsbu...,1.0


In [97]:
dtm_scaled.to_csv('../data/interim/20newsgroup/weighteddtmscaled.csv')
dtm_scaled.head()

,read,24,service,numb,ask,dept,question,news,support,17,...,great,lot,talk,little,happen,claim,doesn,opinion,leave,target
0,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,-0.332306,-0.297651,-0.254025,-0.239241,...,-0.277919,5.889076,-0.249082,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,1.0
1,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,-0.332306,-0.297651,2.721457,-0.239241,...,-0.277919,-0.293101,-0.249082,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,1.0
2,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,1.791602,-0.332306,-0.297651,-0.254025,-0.239241,...,4.392435,-0.293101,3.137025,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,0.0
3,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,-0.332306,-0.297651,-0.254025,-0.239241,...,-0.277919,-0.293101,0.846980,-0.275675,-0.262523,-0.223794,0.719420,-0.27425,-0.260799,1.0
4,-0.310885,-0.224645,-0.227452,-0.245103,-0.290876,-0.209903,1.677027,-0.297651,-0.254025,-0.239241,...,-0.277919,-0.293101,-0.249082,-0.275675,-0.262523,-0.223794,-0.298326,-0.27425,-0.260799,1.0


In [98]:
frame = h2o.upload_file('../data/interim/20newsgroup/weighteddtmscaled.csv',sep=',',header=1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [99]:
frame = frame.drop('C1')

In [100]:
frame['target']=frame['target'].asfactor()
train,test=frame.split_frame(ratios=[.75])

In [118]:
frame.shape

(18846, 201)

0.7641409317627083

In [101]:

optimize_xgb()

xgboost Model Build progress: |                       
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█         

In [102]:
params= {'colsample_bytree': 0.655, 'eta': 0.776, 'learn_rate': 0.88, 'max_depth': 25, 'min_rows': 3, 'ntrees': 298, 'reg_alpha': 0.24, 'reg_lambda': 0.7, 'sample_rate': 0.344}

train

In [103]:
model = H2OXGBoostEstimator(**params,seed=123)

In [104]:
model.train(x=vocab,y='target',training_frame=train)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [105]:
performance = model.model_performance(test)

In [106]:
performance


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.1808733781820403
RMSE: 0.425292109240273
LogLoss: 0.8309084006160609
Mean Per-Class Error: 0.4144868434317665

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,Error,Rate
0,1253.0,448.0,27.0,0.274884,"475 / 1,728"
1,406.0,2419.0,28.0,0.152121,"434 / 2,853"
2,30.0,99.0,29.0,0.816456,129 / 158
3,1689.0,2966.0,84.0,0.219034,"1,038 / 4,739"



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.780966
1,2,0.962650
2,3,1.000000


In [107]:
model.save_mojo('../objects/20newsgroup/modelo_xgb_3class')


'/mnt/c/Users/Raúl/Documents/Doctorado/curso NLP/FINAL-NLP-COURSE/FINAL_NLP_COURSE/Scripts Phd/oboe/objects/20newsgroup/modelo_xgb_3class/XGBoost_model_python_1594800800082_4935.zip'

In [109]:
optimize_rf()

drf Model Build progress: |                           
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█         

In [110]:
params = {'max_depth':25,'min_rows':2,'ntrees':141,'sample_rate':0.417}
rfmodel = H2ORandomForestEstimator(**params,seed=123)


In [111]:
rfmodel.train(x=vocab,y='target',training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [112]:
performance2 = rfmodel.model_performance(test)

In [113]:
performance2


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.17112821143506943
RMSE: 0.41367645743390985
LogLoss: 0.5428082000085279
Mean Per-Class Error: 0.45407889941646046

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,Error,Rate
0,1129.0,592.0,7.0,0.346644,"599 / 1,728"
1,207.0,2646.0,0.0,0.072555,"207 / 2,853"
2,19.0,130.0,9.0,0.943038,149 / 158
3,1355.0,3368.0,16.0,0.201519,"955 / 4,739"



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.798481
1,2,0.971091
2,3,1.000000


In [116]:
rfmodel.save_mojo('../objects/20newsgroup/rf_model_3class')

'/mnt/c/Users/Raúl/Documents/Doctorado/curso NLP/FINAL-NLP-COURSE/FINAL_NLP_COURSE/Scripts Phd/oboe/objects/20newsgroup/rf_model_3class/DRF_model_python_1594800800082_4953.zip'

## Corpus con target original

In [120]:
newdtm =joblib.load('../objects/20newsgroup/newdtmsinoutliers.final')
corpusnormalizado = joblib.load('../data/processed/20newsgroup/corpus_normalizado_sin_outliers.final')
dtm = newdtm.todense()
dtm_scaled = scale(dtm)

In [121]:
dtm_scaled = pd.DataFrame(data=dtm_scaled, columns=vocab) ##revisar que está bien generado, si no ir al paso anterior

In [122]:
corpusnormalizado.head()

,cleaned_text,Topic
0,organization post office carnegie mellon pittsburgh pa line 12 nnt...,1
1,summary seek recommendation vlb video card nntp post host midway.e...,1
2,line 95 nntp post host viktoria.dsv.su.se organization dept comput...,0
3,originator guyd pal500.austin.ibm.com organization ibm austin line...,1
4,organization sophomore mechanical engineer carnegie mellon pittsbu...,1


In [123]:
dtm_scaled['target']=corpusnormalizado.iloc[:,1]

In [124]:
dtm_scaled.head()

,read,24,service,numb,ask,dept,question,news,support,17,...,great,lot,talk,little,happen,claim,doesn,opinion,leave,target
0,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,-0.332330,-0.297618,-0.253975,-0.239238,...,-0.277866,5.889367,-0.249058,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,1
1,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,-0.332330,-0.297618,2.720929,-0.239238,...,-0.277866,-0.293116,-0.249058,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,1
2,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,1.791733,-0.332330,-0.297618,-0.253975,-0.239238,...,4.391607,-0.293116,3.136709,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,0
3,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,-0.332330,-0.297618,-0.253975,-0.239238,...,-0.277866,-0.293116,0.846893,-0.275666,-0.2625,-0.223799,0.719461,-0.274283,-0.260783,1
4,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,1.677149,-0.297618,-0.253975,-0.239238,...,-0.277866,-0.293116,-0.249058,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,1


In [125]:
corpusnormalizado.head()

,cleaned_text,Topic
0,organization post office carnegie mellon pittsburgh pa line 12 nnt...,1
1,summary seek recommendation vlb video card nntp post host midway.e...,1
2,line 95 nntp post host viktoria.dsv.su.se organization dept comput...,0
3,originator guyd pal500.austin.ibm.com organization ibm austin line...,1
4,organization sophomore mechanical engineer carnegie mellon pittsbu...,1


In [126]:
dtm_scaled.to_csv('../data/interim/20newsgroup/weighteddtmscaled.csv')
dtm_scaled.head()

,read,24,service,numb,ask,dept,question,news,support,17,...,great,lot,talk,little,happen,claim,doesn,opinion,leave,target
0,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,-0.332330,-0.297618,-0.253975,-0.239238,...,-0.277866,5.889367,-0.249058,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,1
1,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,-0.332330,-0.297618,2.720929,-0.239238,...,-0.277866,-0.293116,-0.249058,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,1
2,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,1.791733,-0.332330,-0.297618,-0.253975,-0.239238,...,4.391607,-0.293116,3.136709,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,0
3,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,-0.332330,-0.297618,-0.253975,-0.239238,...,-0.277866,-0.293116,0.846893,-0.275666,-0.2625,-0.223799,0.719461,-0.274283,-0.260783,1
4,-0.310883,-0.224645,-0.227476,-0.245143,-0.290845,-0.209919,1.677149,-0.297618,-0.253975,-0.239238,...,-0.277866,-0.293116,-0.249058,-0.275666,-0.2625,-0.223799,-0.298344,-0.274283,-0.260783,1


In [127]:
frame = h2o.upload_file('../data/interim/20newsgroup/weighteddtmscaled.csv',sep=',',header=1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [128]:
frame = frame.drop('C1')

In [129]:
frame['target']=frame['target'].asfactor()
train,test=frame.split_frame(ratios=[.75])

In [130]:
optimize_xgb()

xgboost Model Build progress: |                       
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█         

In [135]:
params= {'colsample_bytree': 0.55, 'eta': 0.41, 'learn_rate': 0.56, 'max_depth': 32, 'min_rows': 9, 'ntrees': 204, 'reg_alpha': 0.96, 'reg_lambda': 0.45, 'sample_rate': 0.336,"nfolds":10}

train

In [136]:
model = H2OXGBoostEstimator(**params,seed=123)

In [137]:
model.train(x=vocab,y='target',training_frame=train)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [138]:
performance = model.model_performance(test)

In [139]:
performance


ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.14380382761862237
RMSE: 0.37921475131991156
LogLoss: 0.5015027961453632
Mean Per-Class Error: 0.20537488877803933
AUC: 0.8739152692778276
AUCPR: 0.8387288085537111
Gini: 0.7478305385556552

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3542644927899043: 


,,0,1,Error,Rate
0,0,1097.0,586.0,0.3482,(586.0/1683.0)
1,1,285.0,2706.0,0.0953,(285.0/2991.0)
2,Total,1382.0,3292.0,0.1864,(871.0/4674.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.354264,0.861372,256.0
1,max f2,0.030503,0.917082,374.0
2,max f0point5,0.738867,0.856483,136.0
3,max accuracy,0.427820,0.814720,235.0
4,max precision,0.999883,1.000000,0.0
5,max recall,0.000176,1.000000,399.0
6,max specificity,0.999883,1.000000,0.0
7,max absolute_mcc,0.427820,0.589860,235.0
8,max min_per_class_accuracy,0.714171,0.791040,145.0
9,max mean_per_class_accuracy,0.662960,0.794625,163.0



Gains/Lift Table: Avg response rate: 63.99 %, avg score: 64.37 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010484,0.999979,1.562688,1.562688,1.000000,0.999989,1.000000,0.999989,0.016382,0.016382,56.268806,56.268806
1,,2,0.020111,0.999944,1.562688,1.562688,1.000000,0.999963,1.000000,0.999976,0.015045,0.031428,56.268806,56.268806
2,,3,0.030167,0.999894,1.562688,1.562688,1.000000,0.999917,1.000000,0.999957,0.015714,0.047141,56.268806,56.268806
3,,4,0.040009,0.999824,1.562688,1.562688,1.000000,0.999860,1.000000,0.999933,0.015379,0.062521,56.268806,56.268806
4,,5,0.050064,0.999721,1.562688,1.562688,1.000000,0.999785,1.000000,0.999903,0.015714,0.078235,56.268806,56.268806
5,,6,0.100128,0.999065,1.556010,1.559349,0.995726,0.999443,0.997863,0.999673,0.077900,0.156135,55.600991,55.934899
6,,7,0.149979,0.997525,1.482206,1.533708,0.948498,0.998365,0.981455,0.999238,0.073888,0.230023,48.220628,53.370811
7,,8,0.200043,0.994909,1.449159,1.512548,0.927350,0.996327,0.967914,0.998510,0.072551,0.302574,44.915944,51.254834
8,,9,0.299957,0.981769,1.402069,1.475748,0.897216,0.989706,0.944365,0.995577,0.140087,0.442661,40.206916,47.574821
9,,10,0.400086,0.947609,1.328953,1.439010,0.850427,0.966839,0.920856,0.988385,0.133066,0.575727,32.895267,43.901008


In [140]:
model.save_mojo('../objects/20newsgroup/modelo_xgb_2class')


'/mnt/c/Users/Raúl/Documents/Doctorado/curso NLP/FINAL-NLP-COURSE/FINAL_NLP_COURSE/Scripts Phd/oboe/objects/20newsgroup/modelo_xgb_2class/XGBoost_model_python_1594800800082_6894.zip'

In [141]:
optimize_rf()

drf Model Build progress: |                           
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█                                                     
█         

In [144]:
params = {'max_depth':38,'min_rows':18,'ntrees':335,'sample_rate':0.35,"nfolds":10}
rfmodel = H2ORandomForestEstimator(**params,seed=123)


In [145]:
rfmodel.train(x=vocab,y='target',training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [146]:
performance2 = rfmodel.model_performance(test)

In [147]:
performance2


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.15272824885324815
RMSE: 0.3908046172363476
LogLoss: 0.4731581266537012
Mean Per-Class Error: 0.2079914729333574
AUC: 0.8747332311849392
AUCPR: 0.9224471035338339
Gini: 0.7494664623698784

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5165927640730777: 


,,0,1,Error,Rate
0,0,1004.0,679.0,0.4034,(679.0/1683.0)
1,1,260.0,2731.0,0.0869,(260.0/2991.0)
2,Total,1264.0,3410.0,0.2009,(939.0/4674.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.516593,0.853304,258.0
1,max f2,0.400983,0.915392,320.0
2,max f0point5,0.655559,0.855439,180.0
3,max accuracy,0.562288,0.809371,232.0
4,max precision,0.978789,1.000000,0.0
5,max recall,0.262596,1.000000,375.0
6,max specificity,0.978789,1.000000,0.0
7,max absolute_mcc,0.562288,0.582547,232.0
8,max min_per_class_accuracy,0.610505,0.790037,207.0
9,max mean_per_class_accuracy,0.589494,0.792009,218.0



Gains/Lift Table: Avg response rate: 63.99 %, avg score: 64.17 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010056,0.953812,1.562688,1.562688,1.000000,0.963038,1.000000,0.963038,0.015714,0.015714,56.268806,56.268806
1,,2,0.020111,0.942640,1.562688,1.562688,1.000000,0.947349,1.000000,0.955194,0.015714,0.031428,56.268806,56.268806
2,,3,0.030167,0.935345,1.562688,1.562688,1.000000,0.939072,1.000000,0.949820,0.015714,0.047141,56.268806,56.268806
3,,4,0.040009,0.927663,1.562688,1.562688,1.000000,0.930883,1.000000,0.945162,0.015379,0.062521,56.268806,56.268806
4,,5,0.050064,0.921238,1.562688,1.562688,1.000000,0.924499,1.000000,0.941012,0.015714,0.078235,56.268806,56.268806
5,,6,0.100128,0.889072,1.529297,1.545993,0.978632,0.905576,0.989316,0.923294,0.076563,0.154798,52.929729,54.599268
6,,7,0.149979,0.854586,1.535861,1.542625,0.982833,0.872188,0.987161,0.906307,0.076563,0.231361,53.586080,54.262502
7,,8,0.200043,0.820721,1.449159,1.519234,0.927350,0.836463,0.972193,0.888827,0.072551,0.303912,44.915944,51.923364
8,,9,0.299957,0.764340,1.422147,1.486894,0.910064,0.792762,0.951498,0.856828,0.142093,0.446005,42.214653,48.689435
9,,10,0.400086,0.708369,1.372361,1.458230,0.878205,0.736564,0.933155,0.826730,0.137412,0.583417,37.236067,45.823031


In [148]:
rfmodel.save_mojo('../objects/20newsgroup/rf_model_2class')

'/mnt/c/Users/Raúl/Documents/Doctorado/curso NLP/FINAL-NLP-COURSE/FINAL_NLP_COURSE/Scripts Phd/oboe/objects/20newsgroup/rf_model_2class/DRF_model_python_1594800800082_10408.zip'

In [ ]:
h2o.shutdown()